In [1]:
import requests
import token_api
from prettyprinter import pprint as pp
import pandas as pd
import pymysql
from datetime import datetime
import datetime

# Ekstrak

In [2]:
url = 'https://api.spoonacular.com/recipes/complexSearch'
headers = {
    'apiKey': token_api.TOKEN
    }
params = {'apiKey': token_api.TOKEN, 'cusine':'indonesia'}
r = requests.get(url, params=params)
df = pd.json_normalize(data=r.json()['results'])

In [3]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
df['date'] = today

In [4]:
df

,id,title,image,imageType,date
0,716426,"Cauliflower, Brown Rice, and Vegetable Fried Rice",https://spoonacular.com/recipeImages/716426-31...,jpg,2022-01-14
1,715594,Homemade Garlic and Basil French Fries,https://spoonacular.com/recipeImages/715594-31...,jpg,2022-01-14
2,715497,Berry Banana Breakfast Smoothie,https://spoonacular.com/recipeImages/715497-31...,jpg,2022-01-14
3,644387,Garlicky Kale,https://spoonacular.com/recipeImages/644387-31...,jpg,2022-01-14
4,716268,African Chicken Peanut Stew,https://spoonacular.com/recipeImages/716268-31...,jpg,2022-01-14
5,716381,Nigerian Snail Stew,https://spoonacular.com/recipeImages/716381-31...,jpg,2022-01-14
6,782601,Red Kidney Bean Jambalaya,https://spoonacular.com/recipeImages/782601-31...,jpg,2022-01-14
7,794349,Broccoli and Chickpea Rice Salad,https://spoonacular.com/recipeImages/794349-31...,jpg,2022-01-14
8,715446,Slow Cooker Beef Stew,https://spoonacular.com/recipeImages/715446-31...,jpg,2022-01-14
9,715415,Red Lentil Soup with Chicken and Turnips,https://spoonacular.com/recipeImages/715415-31...,jpg,2022-01-14


# Transform / Validate

In [5]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing execution")
        return False 

    # # Primary Key Check
    # if pd.Series(df['played_at']).is_unique:
    #     pass
    # else:
    #     raise Exception("Primary Key check is violated")

    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")

    # # Check that all timestamps are of yesterday's date
    # yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    # yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    # timestamps = df["timestamp"].tolist()
    # for timestamp in timestamps:
    #     if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
    #         raise Exception("At least one of the returned songs does not have a yesterday's timestamp")

    return True

In [6]:
if check_if_valid_data(df):
        print("Data valid, proceed to Load stage")

Data valid, proceed to Load stage


# Load

In [7]:
db = pymysql.connect(host=token_api.HOST_NAME_mysql, user=token_api.USERNAME_mysql, password=token_api.PASSWORD_mysql, database=token_api.DBNAME_mysql)
db

In [8]:
cursor = db.cursor()
cursor

In [46]:
# sql = '''use test'''
# cursor.execute(sql)

0

In [9]:
sql = """CREATE TABLE IF NOT EXISTS food (
                    date DATE NOT NULL,
                    food_id TEXT NOT NULL,
                    title TEXT NOT NULL,
                    image TEXT NOT NULL,
                    image_type TEXT NOT NULL
            )"""
cursor.execute(sql)

0

In [10]:
for i,row in df.iterrows():
    insert_into_videos = ("""INSERT INTO food (date, food_id, title, image, image_type)
                            VALUES(%s,%s,%s,%s,%s);""")
    row_to_insert = (row[-1], row[0], row[1], row[2], row[3])
    cursor.execute(insert_into_videos, row_to_insert)

In [12]:
cursor.connection.commit()

In [13]:
db.close()